In [4]:
import os
import json
import math
import pandas as pd
from collections import Counter
from typing import Dict, Any
from openai import OpenAI

# ==============================
# OpenAI 設定與呼叫
# ==============================
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

def get_response(prompt: str, temperature: float = 0.0) -> str:
    """呼叫 OpenAI API，回傳模型輸出（應該是 JSON 字串）"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            stream=False,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"API 調用錯誤: {e}")
        return ""

# ==============================
# 讀取文章（只為了保留順序，不丟進模型）
# ==============================
import pandas as pd

# 讀取 CSV
df = pd.read_csv("./articles_584.csv", encoding="ISO-8859-1")

# 只保留有文章的資料
df = df.dropna(subset=["ARTICLE_TEXT"])

# 確保 id 欄位是整數型態（避免後面拼接出問題）
df["id"] = df["id"].astype(int)

# 建立 dict：用 CSV 裡的 id 當編號
articles = {f"Article_{row['id']}": row["ARTICLE_TEXT"] for _, row in df.iterrows()}

# 列出全部文章編號
all_articles = list(articles.keys())
print("📄 CSV 中的文章編號：")
print(all_articles)  # 先只印前 20 筆
print(f"✅ 總共有 {len(all_articles)} 篇文章")


📄 CSV 中的文章編號：
['Article_1', 'Article_2', 'Article_3', 'Article_4', 'Article_5', 'Article_6', 'Article_8', 'Article_9', 'Article_10', 'Article_11', 'Article_12', 'Article_13', 'Article_14', 'Article_15', 'Article_16', 'Article_17', 'Article_18', 'Article_19', 'Article_20', 'Article_21', 'Article_22', 'Article_23', 'Article_24', 'Article_25', 'Article_26', 'Article_27', 'Article_28', 'Article_30', 'Article_31', 'Article_32', 'Article_33', 'Article_34', 'Article_35', 'Article_36', 'Article_37', 'Article_38', 'Article_39', 'Article_40', 'Article_42', 'Article_43', 'Article_44', 'Article_45', 'Article_46', 'Article_47', 'Article_48', 'Article_49', 'Article_50', 'Article_52', 'Article_53', 'Article_56', 'Article_57', 'Article_58', 'Article_59', 'Article_60', 'Article_61', 'Article_62', 'Article_65', 'Article_66', 'Article_67', 'Article_68', 'Article_69', 'Article_70', 'Article_71', 'Article_72', 'Article_73', 'Article_74', 'Article_75', 'Article_76', 'Article_79', 'Article_80', 'Article_81',

In [7]:
import pandas as pd

# 輸入你的 CSV 檔案名稱
csv_file = "articles_584.csv"

# 假設你知道總共應該有多少篇（例：584）
expected_total = 584

# 讀取 CSV
df = pd.read_csv(csv_file)

# 取得實際出現過的 id（轉成 int）
existing_numbers = set(df["id"].dropna().astype(int))

# 建立完整應有的 ID 集合
# ⚠️ 假設 id 從 1 開始編號
expected_numbers = set(range(1, expected_total + 1))

# 找出缺少的編號
missing = sorted(expected_numbers - existing_numbers)

print(f"✅ 實際文章數量: {len(existing_numbers)} / {expected_total}")
print(f"❌ 缺少的 Article 編號（共 {len(missing)} 篇）:")
print(", ".join(f"Article_{i}" for i in missing))


✅ 實際文章數量: 584 / 584
❌ 缺少的 Article 編號（共 45 篇）:
Article_7, Article_29, Article_41, Article_51, Article_54, Article_55, Article_63, Article_64, Article_77, Article_78, Article_95, Article_106, Article_111, Article_135, Article_138, Article_146, Article_215, Article_229, Article_234, Article_254, Article_261, Article_269, Article_271, Article_287, Article_289, Article_295, Article_353, Article_354, Article_362, Article_372, Article_378, Article_407, Article_408, Article_432, Article_444, Article_447, Article_462, Article_464, Article_472, Article_482, Article_491, Article_497, Article_498, Article_502, Article_509


In [8]:
# ==========================================
# Colab 程式碼區塊 1: 初始化和步驟1
# ==========================================
import json
import os

# Step 1 的提示詞
step1_prompt = """You are a sociology professor with 30 years of experience analyzing Asian racism in the United States.
Your task is to analyze the following article by identifying named entities and classifying them into the appropriate social roles and institutional categories. Focus especially on people or groups reacting to or affected by anti-Asian incidents.

Step 1: Named Entity Recognition and Categorization

1. Identify named entities.
2. Classify them into appropriate social roles or institutional categories.
3. Determine whether each **individual** is **Asian**, **Non-Asian**, or **Cannot be inferred** based on the text.
4. For each entity, include a `"reference"` field that reflects **the exact phrase or wording** used in the article to refer to them.

Please extract all named entities from the text and categorize them according to the following schema:

---

**INDIVIDUALS** (Specific persons or actors representing individual agency)

1. politicians
   - Elected officials acting in an individual capacity.
   - Examples: senators, representatives, mayors, governors

2. professionals
   - Individuals recognized by their expertise or institutional role.
   - Examples: professors, doctors, lawyers, foundation presidents

3. celebrities
   - Public figures in entertainment or sports (e.g., actors, athletes) unless clearly acting in a professional or political role.
   - If overlapping with another role, assign to the more institutionally grounded category.

4. perpetrators
   - Individuals directly identified as committing or responsible for anti-Asian actions.
   - Do not include vague or generalized public unless clearly specified.

5. victims
   - Individuals or racial/ethnic groups explicitly targeted by anti-Asian acts.
   - Examples: “a woman attacked on the subway,” “Japanese Americans during WWII”

6. other_individuals
   - All other named or unnamed individuals who do not fall into the above categories.
   - Includes the general public, community members, business owners, or relatives (e.g., “my mom,” “a neighbor”).

---

**ORGANIZATIONS** (Named institutions or collectives)

1. law_enforcement_agencies
   - Official police or investigative institutions.
   - Examples: Chicago Police Department, FBI, local sheriff’s office

2. government_bodies
   - Government agencies, departments, or offices at any level (local/state/federal).
   - Examples: CDC, Department of Justice, City Council

3. ngo_or_advocacy_groups
   - Civil rights organizations, foundations, or advocacy nonprofits.
   - Examples: Stop AAPI Hate, Robert Wood Johnson Foundation

4. business_entities
   - Named companies, hotels, restaurants, or stores.
   - Examples: Wrap-on Tools, Edgewater Beach Hotel

5. community_groups
   - Named cultural, ethnic, or neighborhood associations.
   - Examples: Chinatown Association, Asian-American Coalition

---

**ETHNICITY INFERENCE RULES:**

- For each **individual**, determine whether they are **Asian**, **Non-Asian**, or **Cannot be inferred**.
- Use contextual clues such as ethnicity indicators, names, or explicit mentions.
- If ethnicity is ambiguous or not stated, return `"Cannot be inferred"`.

---

**ADDITIONAL INSTRUCTIONS:**

- Use `"reference"` to capture how the person/group was referred to in the original article (e.g., `"an 80-year-old woman"`, `"Lee"`, `"the attacker"`).
- Normalize all name variants to a canonical form (e.g., “Dr. Church,” “J. Church,” and “Church” → “Jacqueline Church”).
- If an individual belongs to multiple roles, assign them to the most institutionally specific one (e.g., categorize a lawyer-celebrity as a professional).
- Include only individuals explicitly involved in specific incidents under “victims” and “perpetrators.”
- Do not classify individual police officers or sheriffs as individuals—assign them under law_enforcement_agencies.
- Classify individual owners under “business_actors” and company names under “business_entities.”

---

**Output format (in JSON):**

For all individuals, return an object with "name" and "asian_status" fields.
For all organizations, return an object with "name" and "asian_status": "Not applicable".

```json
{
  "individuals": {
    "politicians": [
      {
        "name": "Tammy Duckworth",
        "reference": "Senator Tammy Duckworth",
        "asian_status": "Asian"
      },
      {
        "name": "Joe Biden",
        "reference": "President Joe Biden",
        "asian_status": "Non-Asian"
      }
    ],
    "professionals": [
      {
        "name": "Julie Morita",
        "reference": "Julie Morita",
        "asian_status": "Asian"
      }
    ],
    "celebrities": [
      {
        "name": "Awkwafina",
        "reference": "Awkwafina",
        "asian_status": "Asian"
      }
    ],
    "perpetrators": [
      {
        "name": "Unknown Attacker",
        "reference": "the attacker",
        "asian_status": "Non-Asian"
      }
    ],
    "victims": [
      {
        "name": "Asian Elderly Woman",
        "reference": "an 80-year-old woman",
        "asian_status": "Asian"
      }
    ],
    "other_individuals": [
      {
        "name": "my mom",
        "reference": "my mom",
        "asian_status": "Asian"
      }
    ]
    
  },
  "organizations": {
    "law_enforcement_agencies": [
      {
        "name": "Chicago Police Department",
        "reference": "Chicago Police Department",
        "asian_status": "Not applicable"
      }
    ],
    "government_bodies": [
      {
        "name": "City Council",
        "reference": "City Council",
        "asian_status": "Not applicable"
      }
    ],
    "ngo_or_advocacy_groups": [
      {
        "name": "Stop AAPI Hate",
        "reference": "Stop AAPI Hate",
        "asian_status": "Not applicable"
      }
    ],
    "business_entities": [
      {
        "name": "Edgewater Beach Hotel",
        "reference": "Edgewater Beach Hotel",
        "asian_status": "Not applicable"
      }
    ],
    "community_groups": [
      {
        "name": "Chinatown Association",
        "reference": "Chinatown Association",
        "asian_status": "Not applicable"
      }
    ]
  }
}


"""

# 呼叫 Step 1

# 建立輸出資料夾
output_dir = "step1_batches"
os.makedirs(output_dir, exist_ok=True)

# 每批文章數量
batch_size = 50
article_items = list(articles.items())
total_articles = len(article_items)
total_batches = (total_articles + batch_size - 1) // batch_size  # 無條件進位

# 逐批處理
for batch_idx in range(0, total_articles, batch_size):
    batch_number = batch_idx // batch_size + 1
    filename = os.path.join(output_dir, f"step1_batch_{batch_number}.json")

    # 如果檔案已經存在，就跳過這批
    if os.path.exists(filename):
        print(f"⏭️ 批次 {batch_number}/{total_batches} 已存在，跳過")
        continue

    print(f"\n🚀 開始處理批次 {batch_number}/{total_batches}...")

    step1_result = {}
    batch = article_items[batch_idx: batch_idx + batch_size]

    for title, content in batch:
        full_prompt = step1_prompt + "\n\nArticle Text:\n" + content
        response = get_response(full_prompt)
        step1_result[title] = response

    # 儲存這一批結果
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(step1_result, f, ensure_ascii=False, indent=2)

    print(f"✅ 批次 {batch_number}/{total_batches} 已完成，儲存至 {filename}")

print("\n🎉 所有批次處理完成！")




🚀 開始處理批次 1/12...
✅ 批次 1/12 已完成，儲存至 step1_batches\step1_batch_1.json

🚀 開始處理批次 2/12...
✅ 批次 2/12 已完成，儲存至 step1_batches\step1_batch_2.json

🚀 開始處理批次 3/12...
✅ 批次 3/12 已完成，儲存至 step1_batches\step1_batch_3.json

🚀 開始處理批次 4/12...
✅ 批次 4/12 已完成，儲存至 step1_batches\step1_batch_4.json

🚀 開始處理批次 5/12...
✅ 批次 5/12 已完成，儲存至 step1_batches\step1_batch_5.json

🚀 開始處理批次 6/12...
✅ 批次 6/12 已完成，儲存至 step1_batches\step1_batch_6.json

🚀 開始處理批次 7/12...
✅ 批次 7/12 已完成，儲存至 step1_batches\step1_batch_7.json

🚀 開始處理批次 8/12...
✅ 批次 8/12 已完成，儲存至 step1_batches\step1_batch_8.json

🚀 開始處理批次 9/12...
✅ 批次 9/12 已完成，儲存至 step1_batches\step1_batch_9.json

🚀 開始處理批次 10/12...
✅ 批次 10/12 已完成，儲存至 step1_batches\step1_batch_10.json

🚀 開始處理批次 11/12...
✅ 批次 11/12 已完成，儲存至 step1_batches\step1_batch_11.json

🚀 開始處理批次 12/12...
✅ 批次 12/12 已完成，儲存至 step1_batches\step1_batch_12.json

🎉 所有批次處理完成！


In [9]:
import os
import glob
import json

step2_prompt = """You are a sociology professor with 30 years of experience analyzing Asian racism in the United States.

**Step 2: Extract Related Sentences**

Based on the named entities identified in **Step 1**, extract **all relevant complete sentences** from the text for each entity.

**For each named entity:**
1. Use the `"reference"` field (the phrase as it appears in the article) to identify relevant sentences.
2. Identify **all complete, verbatim sentences** in the text that mention or describe the entity’s **involvement, action, reaction, statement, or experience** related to anti-Asian hate (directly or indirectly).
3. Do **not paraphrase** or summarize. Use the **exact wording** from the text.
4. If no relevant sentence is found, set `"relevant_sentences": []`.
5. Return results **grouped by entity name**, exactly matching the names used in Step 1.
6. Include the following structured metadata for each entity:
   - `"entity_type"`:  
     - For individuals: the **social role** (e.g., "politician", "celebrity", "victim")  
     - For organizations: the **institutional category** (e.g., "law_enforcement_agency", "ngo_or_advocacy_group")
   - `"asian_status"`:  
     - For individuals: **"Asian"**, **"Non-Asian"**, or **"Cannot be inferred"**  
     - For organizations: Always **"Not applicable"**

**Note:** These sentences will later be used to infer **behavioral reactions** and **emotional responses**, so include **any sentence** that provides context about what the entity did, said, or experienced.

### Output format (JSON):

```json
{
  "Entity Name": {
    "entity_type": "e.g., politician, law_enforcement_agency",
    "asian_status": "Asian / Non-Asian / Cannot be inferred / Not applicable",
    "relevant_sentences": [
      "Sentence 1 from the article.",
      "Sentence 2 from the article."
    ]
  }
}

"""

# 建立輸出資料夾
output_dir = "step2_batches"
os.makedirs(output_dir, exist_ok=True)

# 找出 Step 1 的所有批次檔案
step1_files = sorted(glob.glob("step1_batches/step1_batch_*.json"))

for step1_file in step1_files:
    batch_name = os.path.basename(step1_file).replace("step1_", "step2_")
    output_file = os.path.join(output_dir, batch_name)

    # 如果已經有 Step 2 的結果，就跳過
    if os.path.exists(output_file):
        print(f"⏭️ {batch_name} 已存在，跳過")
        continue

    print(f"\n🚀 開始處理 {step1_file} → {output_file}")

    # 讀取 Step 1 的結果
    with open(step1_file, "r", encoding="utf-8") as f:
        step1_result = json.load(f)

    step2_result = {}

    # 逐篇文章處理
    for title, step1_text in step1_result.items():
        content = articles.get(title, "")  # 從原始文章 dict 拿內容

        full_prompt = (
            step2_prompt +
            f"\n\nStep 1 Results:\n{step1_text}" +
            f"\n\nOriginal Article Text:\n{content}"
        )

        response = get_response(full_prompt)
        step2_result[title] = response
        print(f"✔️ 已處理 {title}")

    # 儲存這一批的 Step 2 結果
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(step2_result, f, ensure_ascii=False, indent=2)

    print(f"✅ {batch_name} 已完成並儲存至 {output_file}")

print("\n🎉 Step 2 全部批次處理完成！")



🚀 開始處理 step1_batches\step1_batch_1.json → step2_batches\step2_batch_1.json
✔️ 已處理 Article_1
✔️ 已處理 Article_2
✔️ 已處理 Article_3
✔️ 已處理 Article_4
✔️ 已處理 Article_5
✔️ 已處理 Article_6
✔️ 已處理 Article_8
✔️ 已處理 Article_9
✔️ 已處理 Article_10
✔️ 已處理 Article_11
✔️ 已處理 Article_12
✔️ 已處理 Article_13
✔️ 已處理 Article_14
✔️ 已處理 Article_15
✔️ 已處理 Article_16
✔️ 已處理 Article_17
✔️ 已處理 Article_18
✔️ 已處理 Article_19
✔️ 已處理 Article_20
✔️ 已處理 Article_21
✔️ 已處理 Article_22
✔️ 已處理 Article_23
✔️ 已處理 Article_24
✔️ 已處理 Article_25
✔️ 已處理 Article_26
✔️ 已處理 Article_27
✔️ 已處理 Article_28
✔️ 已處理 Article_30
✔️ 已處理 Article_31
✔️ 已處理 Article_32
✔️ 已處理 Article_33
✔️ 已處理 Article_34
✔️ 已處理 Article_35
✔️ 已處理 Article_36
✔️ 已處理 Article_37
✔️ 已處理 Article_38
✔️ 已處理 Article_39
✔️ 已處理 Article_40
✔️ 已處理 Article_42
✔️ 已處理 Article_43
✔️ 已處理 Article_44
✔️ 已處理 Article_45
✔️ 已處理 Article_46
✔️ 已處理 Article_47
✔️ 已處理 Article_48
✔️ 已處理 Article_49
✔️ 已處理 Article_50
✔️ 已處理 Article_52
✔️ 已處理 Article_53
✔️ 已處理 Article_56
✅ step2_batch_1.json 已完成並儲存至 ste

In [11]:
import os
import json

def rerun_incomplete_step2(step1_file, step2_file):
    # 讀取 Step1 的結果
    with open(step1_file, "r", encoding="utf-8") as f:
        step1_result = json.load(f)

    # 如果 step2_file 已存在，先讀取；否則新建
    if os.path.exists(step2_file):
        with open(step2_file, "r", encoding="utf-8") as f:
            step2_result = json.load(f)
    else:
        step2_result = {}

    updated = False

    for title, step1_text in step1_result.items():
        # 只處理空的 or 缺失的
        if not step2_result.get(title):
            content = articles.get(title, "")  # 原始文章
            full_prompt = (
                step2_prompt +
                f"\n\nStep 1 Results:\n{step1_text}" +
                f"\n\nOriginal Article Text:\n{content}"
            )
            response = get_response(full_prompt)
            step2_result[title] = response
            print(f"🔄 重新處理 {title}")
            updated = True

    if updated:
        with open(step2_file, "w", encoding="utf-8") as f:
            json.dump(step2_result, f, ensure_ascii=False, indent=2)
        print(f"✅ 已更新 {step2_file}")
    else:
        print("👌 沒有需要補跑的文章，全數完成")

# 使用範例
rerun_incomplete_step2(
    step1_file="step1_batches/step1_batch_9.json",
    step2_file="step2_batches/step2_batch_9.json"
)


🔄 重新處理 Article_485
🔄 重新處理 Article_486
🔄 重新處理 Article_487
🔄 重新處理 Article_488
🔄 重新處理 Article_489
🔄 重新處理 Article_490
✅ 已更新 step2_batches/step2_batch_9.json


In [12]:
# -*- coding: utf-8 -*-
import os
import re
import glob
import json
from collections import Counter
from typing import Dict, Any
from openai import OpenAI


# ==============================
# 工具：JSON 寬鬆解析 + 句子正規化
# ==============================
def strip_code_fences(s: str) -> str:
    if not isinstance(s, str):
        return s
    s = s.strip()
    m = re.search(r"```(?:json)?\s*(.*?)```", s, flags=re.S | re.I)
    if m:
        return m.group(1).strip()
    return s

def parse_json_loose(s: str):
    if not isinstance(s, str):
        return None
    s = strip_code_fences(s).strip()
    if not s:
        return None
    try:
        return json.loads(s)
    except Exception:
        pass
    m = re.search(r"\{.*\}", s, flags=re.S)
    if m:
        try:
            return json.loads(m.group(0))
        except Exception:
            return None
    return None

def to_text(x: Any) -> str:
    if x is None:
        return ""
    if isinstance(x, str):
        return x
    if isinstance(x, list):
        return "\n".join(str(item) for item in x)
    if isinstance(x, dict):
        if "relevant_sentences" in x:
            return to_text(x["relevant_sentences"])
        return "\n".join(to_text(v) for v in x.values())
    return str(x)

def parse_model_json(s: str, default: Dict[str, Any]) -> Dict[str, Any]:
    if not isinstance(s, str) or not s.strip():
        return default
    s2 = strip_code_fences(s)
    try:
        return json.loads(s2)
    except Exception:
        m = re.search(r"\{.*\}", s2, flags=re.S)
        if m:
            try:
                return json.loads(m.group(0))
            except Exception:
                return default
        return default

# ==============================
# Step2 結果正規化
# ==============================
def normalize_step2_result(title: str, raw_obj) -> Dict[str, Dict[str, str]]:
    if isinstance(raw_obj, dict):
        if all(isinstance(v, dict) for v in raw_obj.values()):
            out = {}
            for k, v in raw_obj.items():
                out[k] = {
                    "entity_type": v.get("entity_type", ""),
                    "asian_status": v.get("asian_status", ""),
                    "relevant_sentences": to_text(v.get("relevant_sentences"))
                }
            return out
    if isinstance(raw_obj, str):
        parsed = parse_json_loose(raw_obj)
        if parsed:
            return normalize_step2_result(title, parsed)
    return {}

# ==============================
# Prompts
# ==============================
def build_gate_prompt(step2_sentences: str) -> str:
    return f"""You are a sociology professor analyzing reactions to anti-Asian hate.

Task: Decide if the extracted sentences show any OBSERVABLE reaction (action or inaction) to anti-Asian hate by the entity.

Rules:
- Observable = concrete action/inaction or explicit public stance (e.g., speaking up, condemning, organizing, reporting, policy ask, government action, refusing to act).
- Pure emotions/concerns are NOT reactions.
- Pure incident descriptions are NOT reactions.
- Use ONLY the exact `relevant_sentences`.

Output JSON:
{{
  "has_reaction": "yes" | "no",
  "evidence": "Exact sentence(s) from relevant_sentences (or empty if no)."
}}

relevant_sentences:
{step2_sentences}
"""

def build_classifier_prompt(entity_type: str, asian_status: str, step2_sentences: str) -> str:
    return f"""You are a sociology professor with 30 years of experience analyzing anti-Asian racism.

Task: Classify the entity’s REACTION strictly using the Reaction Concept Tree. Use ONLY `relevant_sentences` as evidence. Do NOT infer emotions. Do NOT paraphrase.

Reaction Concept Tree:
- Support Asian Americans
  - Attending marches/rallies
  - Speaking up on social media
  - Calling for being united
  - Educating students
  - Fostering conversations about anti-Asian hate
  - Hiring security guards
  - Providing shopkeepers with air horns
  - Rewarding the public to report the info about the suspects
- Advocacy/take actions for changes
- Politicians initiated anti-Asian hate and racism
- Undermining human rights
- Color blind/minimizing racism
- Youth as not an excuse
- Videotaping/confronting harasser/attacker
- Sex (sexual) addiction
- Religion as a reason
- Feeling hopeless or support AAPI being not enough
- Not confronting attacker/harasser or not reporting
- Useless law enforcement
  - Did not take a report on Anti-Asian hate crime
  - Did not often patrol the streets
- Government takes actions to stop AAPI hate
  - Installing hotlines
  - Launching a hate-crime task force
  - Increasing patrols
  - Organizing a town hall

Strict Rules:
1) Pure concerns/worries ≠ reaction; return "Cannot be inferred".
2) Arrests/charges/prosecutions ⇒ “Government takes actions…”.
3) Explicit condemnation ⇒ “Support Asian Americans”.
4) If no clear reaction, return "Cannot be inferred".
5) Do NOT invent labels.
6) Always choose the most specific subcategory.

entity_type: {entity_type}
asian_status: {asian_status}

relevant_sentences:
{step2_sentences}

Output JSON:
{{
  "reaction": "<one label from the tree or 'Cannot be inferred'>",
  "reaction_reason": "Exact sentence(s) from relevant_sentences"
}}
"""

# ==============================
# 主流程：從 Step2 → Step3
# ==============================
def run_step3_from_step2(step2_dir="step2_batches", step3_dir="step3_batches"):
    os.makedirs(step3_dir, exist_ok=True)
    step2_files = sorted(glob.glob(os.path.join(step2_dir, "step2_batch_*.json")))
    total_batches = len(step2_files)

    for idx, step2_file in enumerate(step2_files, start=1):
        batch_name = os.path.basename(step2_file).replace("step2_", "step3_")
        out_path = os.path.join(step3_dir, batch_name)

        # 斷點續跑
        if os.path.exists(out_path):
            print(f"⏭️ 批次 {idx}/{total_batches} {batch_name} 已存在，跳過")
            continue

        print(f"\n🚀 開始處理批次 {idx}/{total_batches}: {step2_file}")

        with open(step2_file, "r", encoding="utf-8") as f:
            step2_batch_result = json.load(f)

        step3_batch_result = {}
        gate_stats = Counter()
        label_stats = Counter()

        for title, raw in step2_batch_result.items():
            entities = normalize_step2_result(title, raw)
            entity_outputs = {}

            if not entities:
                step3_batch_result[title] = entity_outputs
                continue

            for entity, meta in entities.items():
                entity_type = meta.get("entity_type", "")
                asian_status = meta.get("asian_status", "")
                relevant_sentences = to_text(meta.get("relevant_sentences", "")).strip()

                # --- 3A: Gate ---
                gate_prompt = build_gate_prompt(relevant_sentences)
                gate_resp = get_response(gate_prompt, temperature=0.0)
                gate_json = parse_model_json(gate_resp, default={"has_reaction": "no", "evidence": ""})

                has_reaction = str(gate_json.get("has_reaction", "no")).lower()
                gate_stats[has_reaction] += 1

                if has_reaction != "yes":
                    out = {
                        "entity_type": entity_type,
                        "asian_status": asian_status,
                        "reaction": "Cannot be inferred",
                        "reaction_reason": ""
                    }
                    entity_outputs[entity] = out
                    label_stats["Cannot be inferred"] += 1
                    continue

                # --- 3B: Classifier ---
                cls_prompt = build_classifier_prompt(entity_type, asian_status, relevant_sentences)
                cls_resp = get_response(cls_prompt, temperature=0.0)
                cls_json = parse_model_json(cls_resp, default={"reaction": "Cannot be inferred", "reaction_reason": ""})

                label = cls_json.get("reaction", "Cannot be inferred")
                reason = cls_json.get("reaction_reason", "")

                out = {
                    "entity_type": entity_type,
                    "asian_status": asian_status,
                    "reaction": label,
                    "reaction_reason": reason
                }
                entity_outputs[entity] = out
                label_stats[label] += 1

            step3_batch_result[title] = entity_outputs

        # 儲存
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(step3_batch_result, f, ensure_ascii=False, indent=2)

        print(f"✅ 批次 {idx}/{total_batches} 已完成並儲存至 {out_path}")
        print("   Gate stats:", dict(gate_stats))
        print("   Label stats:", dict(label_stats))

# ==============================
# 執行
# ==============================
if __name__ == "__main__":
    run_step3_from_step2("step2_batches", "step3_batches")



🚀 開始處理批次 1/12: step2_batches\step2_batch_1.json
✅ 批次 1/12 已完成並儲存至 step3_batches\step3_batch_1.json
   Gate stats: {'no': 426, 'yes': 174}
   Label stats: {'Cannot be inferred': 466, 'Support Asian Americans': 86, 'Government takes actions to stop AAPI hate': 30, 'Advocacy/take actions for changes': 4, 'Attending marches/rallies': 1, 'Politicians initiated anti-Asian hate and racism': 4, 'Color blind/minimizing racism': 1, 'Fostering conversations about anti-Asian hate': 2, 'Useless law enforcement': 2, 'Hiring security guards': 2, 'Educating students': 1, 'Videotaping/confronting harasser/attacker': 1}

🚀 開始處理批次 2/12: step2_batches\step2_batch_10.json
✅ 批次 2/12 已完成並儲存至 step3_batches\step3_batch_10.json
   Gate stats: {'no': 285, 'yes': 131}
   Label stats: {'Cannot be inferred': 296, 'Support Asian Americans': 56, 'Government takes actions to stop AAPI hate': 51, 'Attending marches/rallies': 2, 'Advocacy/take actions for changes': 5, 'Politicians initiated anti-Asian hate and racism':

In [14]:
# -*- coding: utf-8 -*-
import os
import glob
import json
from openai import OpenAI
from collections import Counter


# ==============================
# Step 4 Prompt
# ==============================
step4_prompt = """You are a sociology professor with 30 years of experience analyzing racial dynamics and anti-Asian racism in the United States.

### Step 4: Infer **Emotions and Their Intensity**

Your task is to analyze the **extracted sentences** from **Step 2** and infer each entity's **emotional stance** toward anti-Asian hate.

You will also receive metadata from Step 2, including:
- `entity_type`: for individuals use their social role; for organizations use institutional category.
- `asian_status`: "Asian", "Non-Asian", "Cannot be inferred", or "Not applicable"

Use only the exact `relevant_sentences` from Step 2 as your source — do NOT paraphrase or add your own wording.

---

## Emotion Concept Tree
- Love 
- Joy 
- Anger 
- Sadness 
- Fear 
- Surprise 

---

## Instructions
1. Focus only on emotions — do NOT infer actions.
2. If no emotion is expressed, output `"emotion": "Cannot be inferred".
3. If multiple emotions appear, list multiple objects.
4. Use the exact sentence(s) as `"emotion_reason"`.

---

## Output format
{
  "Entity Name": {
    "entity_type": "...",
    "asian_status": "...",
    "emotions": [
      {
        "emotion": "deepest matched term or Cannot be inferred",
        "emotion_reason": "Exact sentence(s)"
      }
    ]
  }
}
"""

# ==============================
# 主流程 Step2 → Step4
# ==============================
def run_step4_from_step2(step2_dir="step2_batches", step4_dir="step4_batches"):
    os.makedirs(step4_dir, exist_ok=True)
    step2_files = sorted(glob.glob(os.path.join(step2_dir, "step2_batch_*.json")))
    total_batches = len(step2_files)

    for idx, step2_file in enumerate(step2_files, start=1):
        batch_name = os.path.basename(step2_file).replace("step2_", "step4_")
        out_path = os.path.join(step4_dir, batch_name)

        # 斷點續跑
        if os.path.exists(out_path):
            print(f"⏭️ 批次 {idx}/{total_batches} {batch_name} 已存在，跳過")
            continue

        print(f"\n🚀 開始處理批次 {idx}/{total_batches}: {step2_file}")

        with open(step2_file, "r", encoding="utf-8") as f:
            step2_batch_result = json.load(f)

        step4_batch_result = {}
        emo_stats = Counter()

        for title, step2_text in step2_batch_result.items():
            full_prompt = (
                step4_prompt +
                f"\n\nStep 2 Results (Extracted Sentences):\n{step2_text}"
            )

            response = get_response(full_prompt)
            step4_batch_result[title] = response
            emo_stats["done"] += 1

        # 儲存
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(step4_batch_result, f, ensure_ascii=False, indent=2)

        print(f"✅ 批次 {idx}/{total_batches} 已完成並儲存至 {out_path}")
        print("   Stats:", dict(emo_stats))

# ==============================
# 執行
# ==============================
if __name__ == "__main__":
    run_step4_from_step2("step2_batches", "step4_batches")



🚀 開始處理批次 1/12: step2_batches\step2_batch_1.json
✅ 批次 1/12 已完成並儲存至 step4_batches\step4_batch_1.json
   Stats: {'done': 50}

🚀 開始處理批次 2/12: step2_batches\step2_batch_10.json
✅ 批次 2/12 已完成並儲存至 step4_batches\step4_batch_10.json
   Stats: {'done': 50}

🚀 開始處理批次 3/12: step2_batches\step2_batch_11.json
✅ 批次 3/12 已完成並儲存至 step4_batches\step4_batch_11.json
   Stats: {'done': 50}

🚀 開始處理批次 4/12: step2_batches\step2_batch_12.json
✅ 批次 4/12 已完成並儲存至 step4_batches\step4_batch_12.json
   Stats: {'done': 34}

🚀 開始處理批次 5/12: step2_batches\step2_batch_2.json
✅ 批次 5/12 已完成並儲存至 step4_batches\step4_batch_2.json
   Stats: {'done': 50}

🚀 開始處理批次 6/12: step2_batches\step2_batch_3.json
✅ 批次 6/12 已完成並儲存至 step4_batches\step4_batch_3.json
   Stats: {'done': 50}

🚀 開始處理批次 7/12: step2_batches\step2_batch_4.json
✅ 批次 7/12 已完成並儲存至 step4_batches\step4_batch_4.json
   Stats: {'done': 50}

🚀 開始處理批次 8/12: step2_batches\step2_batch_5.json
✅ 批次 8/12 已完成並儲存至 step4_batches\step4_batch_5.json
   Stats: {'done': 50}

🚀 開始處理批次 

In [18]:
import os
import glob
import json
import pandas as pd

def merge_step3_with_id(input_dir="step3_batches", prefix="step3_batch_", 
                        output_json="step3_all.json", output_csv="step3_all.csv"):
    batch_files = sorted(glob.glob(os.path.join(input_dir, f"{prefix}*.json")))

    merged_result = {}
    for file in batch_files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
            merged_result.update(data)

    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(merged_result, f, ensure_ascii=False, indent=2)

    rows = []
    idx = 1
    for article_id, entities in merged_result.items():
        for entity, meta in entities.items():
            rows.append({
                "reaction_id": f"reaction_{idx}",
                "article_id": article_id,
                "entity": entity,
                "entity_type": meta.get("entity_type", ""),
                "asian_status": meta.get("asian_status", ""),
                "reaction": meta.get("reaction", ""),
                "reaction_reason": meta.get("reaction_reason", "")
            })
            idx += 1

    df = pd.DataFrame(rows)
    df.to_csv(output_csv, index=False, encoding="utf-8-sig")

    print(f"✅ Step 3 已合併 {len(batch_files)} 個批次檔 → {output_csv}，共 {len(df)} 筆")

if __name__ == "__main__":
    merge_step3_with_id(
        input_dir="step3_batches",
        prefix="step3_batch_",
        output_json="step3_all.json",
        output_csv="step3_all.csv"
    )


✅ Step 3 已合併 12 個批次檔 → step3_all.csv，共 5693 筆


In [1]:
import os
import glob
import json
import pandas as pd
import re

def parse_json_loose(s: str):
    """嘗試從字串裡解析 JSON"""
    if not isinstance(s, str):
        return None
    s = s.strip()
    if not s:
        return None
    try:
        return json.loads(s)
    except Exception:
        # 嘗試抓第一個 {...}
        m = re.search(r"\{.*\}", s, flags=re.S)
        if m:
            try:
                return json.loads(m.group(0))
            except Exception:
                return None
    return None

def merge_step4_with_id(input_dir="step4_batches", prefix="step4_batch_", 
                        output_json="step4_all.json", output_csv="step4_all.csv"):
    batch_files = sorted(glob.glob(os.path.join(input_dir, f"{prefix}*.json")))
    all_articles = {}

    for file in batch_files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)

        # ⚠️ 不用 update，逐篇展開
        for article_id, raw in data.items():
            if isinstance(raw, dict):
                all_articles[article_id] = raw
            elif isinstance(raw, str):
                parsed = parse_json_loose(raw)
                if isinstance(parsed, dict):
                    all_articles[article_id] = parsed

    # 存 JSON
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(all_articles, f, ensure_ascii=False, indent=2)

    # 轉成 CSV
    rows = []
    idx = 1
    for article_id, entities in all_articles.items():
        for entity, meta in entities.items():
            emotions = meta.get("emotions", [])
            
            # 統一處理格式
            if isinstance(emotions, str):
                emotions = [{
                    "emotion": emotions,
                    "emotion_path": None,
                    "emotion_reason": ""
                }]
            elif not isinstance(emotions, list):
                emotions = []
    
            for emo in emotions:
                if not isinstance(emo, dict):  # 再保險一次
                    emo = {"emotion": str(emo), "emotion_path": None, "emotion_reason": ""}
                rows.append({
                    "emotion_id": f"emotion_{idx}",
                    "article_id": article_id,
                    "entity": entity,
                    "entity_type": meta.get("entity_type", ""),
                    "asian_status": meta.get("asian_status", ""),
                    "emotion": emo.get("emotion", ""),
                    "emotion_path": emo.get("emotion_path", ""),
                    "emotion_reason": emo.get("emotion_reason", "")
                })
                idx += 1

    df = pd.DataFrame(rows)
    df.to_csv(output_csv, index=False, encoding="utf-8-sig")

    print(f"✅ Step 4 已合併 {len(batch_files)} 個批次檔 → {output_csv}，共 {len(df)} 筆")


if __name__ == "__main__":
    merge_step4_with_id(
        input_dir="step4_batches",
        prefix="step4_batch_",
        output_json="step4_all.json",
        output_csv="step4_all.csv"
    )


✅ Step 4 已合併 12 個批次檔 → step4_all.csv，共 8011 筆


# 重分 entity_type

In [ ]:
import pandas as pd

# === 1. 讀取原始資料 ===
df = pd.read_csv("step4_all_with_date.csv")  # 換成你的檔案名稱

# === 2. 定義標準化對應表 ===
entity_type_mapping = {
    # Individuals
    "victim": "victims",
    "victims": "victims",
    "perpetrator": "perpetrators",
    "perpetrators": "perpetrators",
    "politician": "politicians",
    "politicians": "politicians",
    "professional": "professionals",
    "professionals": "professionals",
    "celebrity": "celebrities",
    "musician": "celebrities",
    "actor": "celebrities",
    "actress": "celebrities",
    "journalist": "professionals",
    "reporter": "professionals",
    "professor": "professionals",
    "student": "professionals",
    "educator": "professionals",
    "scholar": "professionals",
    "adjunct instructor": "professionals",
    "attorney": "professionals",
    "director": "professionals",
    "city_manager": "professionals",
    "sociology professor": "professionals",
    "editor": "professionals",
    "deputy inspector": "law_enforcement_agencies",
    "police_officer": "law_enforcement_agencies",
    "police spokesperson": "law_enforcement_agencies",
    "family_member": "other_individuals",
    "friend": "other_individuals",
    "witness": "other_individuals",
    "individual": "other_individuals",
    "individuals": "other_individuals",
    "general public": "other_individuals",
    "general_public": "other_individuals",
    "social_circle": "other_individuals",
    "community_activist": "other_individuals",
    "community_leader": "other_individuals",
    "organizer": "other_individuals",
    "community organizer": "other_individuals",
    "other individual": "other_individuals",
    "rally organizer": "other_individuals",
    "activist": "other_individuals",
    "supporter": "other_individuals",
    "co-host": "celebrities",
    "artist": "celebrities",
    "former assistant district attorney": "professionals",
    "official": "professionals",
    "non-Asian": "other_individuals",
    "youth coordinator": "professionals",
    "school_board_member": "professionals",
    "Dean": "professionals",
    "community leader": "other_individuals",
    "government body": "government_bodies",

    # Organizations
    "law_enforcement_agency": "law_enforcement_agencies",
    "law_enforcement_agencies": "law_enforcement_agencies",
    "government_body": "government_bodies",
    "government_bodies": "government_bodies",
    "ngo_or_advocacy_group": "ngo_or_advocacy_groups",
    "ngo_or_advocacy_groups": "ngo_or_advocacy_groups",
    "business_entity": "business_entities",
    "business_entities": "business_entities",
    "community_group": "community_groups",
    "community_groups": "community_groups",
    "educational_institution": "government_bodies",  # 假設為正式機構

    # Fallback
    "other": "other_individuals",
    "other_individual": "other_individuals",
    "other_individuals": "other_individuals",
    "group": "unknown",
    # "Cannot be inferred": "unknown",
}

# === 3. 替換 entity_type 欄位（直接覆蓋）===
df["entity_type"] = df["entity_type"].map(entity_type_mapping).fillna(df["entity_type"])

# === 4. 輸出成新檔案 ===
df.to_csv("step4_all_with_date.csv", index=False)
print("✅ finish")


# 重分 emotion

In [ ]:
import pandas as pd

# 載入 CSV
df = pd.read_csv("step4_all_with_date.csv")

# 定義 emotion 對照表（細分類 → 六大情緒，全小寫）
emotion_map = {
    # love
    "love": "love",
    "support": "love", "solidarity": "love",
    "empathy": "love", "compassion": "love",
    "recognition": "love", "gratitude": "love",
    "appreciation": "love", "encouragement": "love",
    "affection": "love", "lust": "love", "longing": "love",

    # joy
    "joy": "joy",
    "confidence": "joy", "optimism": "joy", "empowerment": "joy",
    "cheerfulness": "joy", "zest": "joy", "contentment": "joy",
    "pride": "joy", "relief": "joy",

    # anger
    "anger": "anger",
    "outrage": "anger", "defiance": "anger", "responsibility": "anger",
    "irritation": "anger", "exasperation": "anger", "rage": "anger",
    "disgust": "anger", "envy": "anger", "determination": "anger",
    "urgency": "anger", "frustration": "anger",

    # sadness
    "sadness": "sadness",
    "worry": "sadness", "resignation": "sadness", "regret": "sadness",
    "mixed emotions": "sadness", "dismay": "sadness",
    "disquiet": "sadness", "disturbance": "sadness",
    "guilt": "sadness",
    "suffering": "sadness", "disappointment": "sadness", "shame": "sadness",
    "neglect": "sadness", "sympathy": "sadness", "heartbreak": "sadness",
    "pain": "sadness", "grief": "sadness", "grieving": "sadness",
    "hurt": "sadness", "loneliness": "sadness", "despondency": "sadness",
    "helplessness": "sadness", "exhaustion": "sadness",

    # fear
    "fear": "fear",
    "terror": "fear", "doubt": "fear",
    "alarm": "fear", "anxiety": "fear", "insecurity": "fear",
    "panic": "fear", "dread": "fear", "overwhelming": "fear",
    "overwhelmed": "fear", "horror": "fear", "shock": "fear",

    # surprise
    "surprise": "surprise",
    "confusion": "surprise", "lightbulb moment": "surprise",
    "amazement": "surprise", "wonder": "surprise"
}

def map_emotions(emotion_str):
    """把情緒收斂成六大基本情緒，其他歸為 cannot be inferred，全小寫"""
    if pd.isna(emotion_str):
        return "cannot be inferred"
    emotions = [e.strip().lower() for e in emotion_str.split("|")]
    mapped = [emotion_map.get(e, "cannot be inferred") for e in emotions]
    mapped = list(dict.fromkeys(mapped))  # 去重但保留順序
    return " | ".join(mapped)

# 建立新的欄位
df["emotion"] = df["emotion"].apply(map_emotions)

# 輸出結果
df.to_csv("step4_all_with_date.csv", index=False)
print("✅ 已完成：emotion 全部轉成小寫 (love, joy, anger, sadness, fear, surprise, cannot be inferred)")
